In [77]:
!pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 115.9 kB/s eta 0:00:00 kB/s eta 0:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 833.8 kB/s eta 0:00:00m eta 0:00:010:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 2.1 MB/s eta 0:00:002.4 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 2.9 MB/s eta 0:00:00m eta 0:00:010:00:01


In [78]:
from datasets import load_dataset
from openai import OpenAI
from  dotenv import load_dotenv
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from typing import List
load_dotenv()
client = OpenAI()


In [36]:
ds_core = load_dataset("BASF-AI/ChemHotpotQARetrieval", "default")
ds_corpus = load_dataset("BASF-AI/ChemHotpotQARetrieval", "corpus")
ds_queries = load_dataset("BASF-AI/ChemHotpotQARetrieval", "queries")

Using the latest cached version of the dataset since BASF-AI/ChemHotpotQARetrieval couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /Users/kevinmaikjablonka/.cache/huggingface/datasets/BASF-AI___chem_hotpot_qa_retrieval/default/0.0.0/1840e8a5ac6ec752bbdd97d543ead0189bc7c25b (last modified on Fri Jul 25 22:36:13 2025).
Using the latest cached version of the dataset since BASF-AI/ChemHotpotQARetrieval couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'corpus' at /Users/kevinmaikjablonka/.cache/huggingface/datasets/BASF-AI___chem_hotpot_qa_retrieval/corpus/0.0.0/1840e8a5ac6ec752bbdd97d543ead0189bc7c25b (last modified on Fri Jul 25 23:04:49 2025).
Using the latest cached version of the dataset since BASF-AI/ChemHotpotQARetrieval couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'queries' at /Users/kevinmaikjablonka/.cache/huggingface/datasets/BASF-AI___chem_hotpot

In [35]:
def get_embedding(text):
    response = client.embeddings.create(
        input=text,
        model="text-embedding-3-small"
    )
    return response.data[0].embedding

In [6]:
corpus_embeddings = [get_embedding(text) for text in ds_corpus['corpus']['text']]

In [10]:
def save_embeddings(embeddings, filename):
    np.save(filename, embeddings)

In [11]:
save_embeddings(corpus_embeddings, "corpus_embeddings.npy")

In [37]:
queries_embeddings  = [get_embedding(text) for text in ds_queries['queries']['text']]

In [38]:
save_embeddings(queries_embeddings, "queries_embeddings.npy")

In [ ]:
corpus_embeddings = np.load('corpus_embeddings.npy')
queries_embeddings = np.load('cqueries_embeddings.npy')

In [79]:


def compute_dcg_at_k(relevances: List[float], k: int) -> float:
    """
    Compute Discounted Cumulative Gain at rank k.
    
    Args:
        relevances: List of relevance scores for ranked items
        k: Rank cutoff
    
    Returns:
        DCG@k score
    """
    relevances = np.array(relevances[:k])
    if len(relevances) == 0:
        return 0.0
    
    # DCG formula: sum((2^rel - 1) / log2(rank + 1)) for rank 1 to k
    ranks = np.arange(1, len(relevances) + 1)
    dcg = np.sum((2**relevances - 1) / np.log2(ranks + 1))
    return dcg

def compute_ndcg_at_k(relevances: List[float], k: int) -> float:
    """
    Compute Normalized Discounted Cumulative Gain at rank k.
    
    Args:
        relevances: List of relevance scores for ranked items
        k: Rank cutoff
    
    Returns:
        NDCG@k score
    """
    # Actual DCG
    dcg = compute_dcg_at_k(relevances, k)
    
    # Ideal DCG (sort relevances in descending order)
    ideal_relevances = sorted(relevances, reverse=True)
    idcg = compute_dcg_at_k(ideal_relevances, k)
    
    if idcg == 0:
        return 0.0
    
    return dcg / idcg

In [41]:
queries_embeddings = np.array(queries_embeddings)
corpus_embeddings = np.array(corpus_embeddings)

In [42]:
ds_core['train'].to_pandas()

,query-id,corpus-id,score
0,5ae75ace5542997b22f6a6f3,3395507,1.0
1,5adc2d51554299438c868d69,322632,1.0
2,5ae0f0f95542990adbacf6e8,18459715,1.0
3,5abeb1a45542990832d3a0b4,1049937,1.0
4,5ac313e75542995ef918c0f7,40367176,1.0
...,...,...,...
182,5a7339655542991f29ee2d68,23247967,1.0
183,5a7c8f705542990527d554d0,35760918,1.0
184,5a84a20b5542991dd0999d50,62153,1.0
185,5a7c9ea055429907fabeefcc,1374187,1.0


In [85]:
results = []
corpus_df = ds_corpus['corpus'].to_pandas()
k = 10
corpus_ids = corpus_df['_id'].tolist()
for i, row in ds_core['train'].to_pandas().iterrows():
    query = row['query-id']
    ground_truth = row['corpus-id']
    ground_truths = []

    similarities = []  
    for j, corpus_row in corpus_df.iterrows():
        similarity = np.dot(queries_embeddings[i], corpus_embeddings[j])
        similarities.append(similarity)
        ground_truths.append(1 if corpus_row['_id'] == ground_truth else 0)
    
    top_indices = np.argsort(similarities)[::-1][:k]
    top_corpus_ids = [corpus_ids[idx] for idx in top_indices]
    
    # Create relevance scores based on ground truth
    relevant_corpus_ids = {ground_truth}
    relevances = [1.0 if corpus_id in relevant_corpus_ids else 0.0 
                    for corpus_id in top_corpus_ids]
                    
    results.append({
        'query-id': query,
        'ndcg_at_10': compute_ndcg_at_k(relevances, k),
    })


In [86]:
results_frame = pd.DataFrame(results)

In [87]:
results_frame['ndcg_at_10'].mean()

0.44215238908656435